In [3]:
import numpy as np
import cv2 as cv
import glob

# define the dimensions of checkerboard
# 取棋盤行列-1，因為是找交會點
checkerboard = (12, 8)

# termination criteria
# 每當運行30次演算法迭代或達到epsilon = 0.001的精度時，就停止演算法並返回答案。
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# np.zero()是創建空白數組，np.mgrid()是創建多維數組
objp = np.zeros((checkerboard[0]*checkerboard[1],3), np.float32)
objp[:,:2] = np.mgrid[0:checkerboard[0],0:checkerboard[1]].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in image plane.
imgpointsR = [] # 2d points in image plane.

# Extracting path of individual image stored in a given directory
# 分別放在兩個資料夾中
imagesLeft = sorted(glob.glob('left\\*.jpg'))
imagesRight = sorted(glob.glob('right\\*.jpg'))

for imgLeft, imgRight in zip(imagesLeft, imagesRight):
    
    # 讀取圖片
    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
    # 色彩轉換 RGB轉灰階
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)            

    # Find the chess board corners
    # 尋找棋盤，回傳角點座標
    retL, cornersL = cv.findChessboardCorners(grayL, checkerboard, None)
    retR, cornersR = cv.findChessboardCorners(grayR, checkerboard, None)
     
    # If found, add object points, image points (after refining them)
    if retL and retR == True:
        objpoints.append(objp)
        
        # 尋找亞像素角點，優化棋盤角點
        # refine pixel coordinates
        corners2L = cv.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(corners2L)
        
        corners2R = cv.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(corners2R)
    
    # Draw and display the corners
    cv.drawChessboardCorners(imgL, checkerboard, corners2L, retL)
    cv.imshow('img left', imgL)
    cv.drawChessboardCorners(imgR, checkerboard, corners2R, retR)
    cv.imshow('img right', imgR)
    cv.waitKey(500)
    
cv.destroyAllWindows()

In [4]:
# 校準相機
# returns reprojection error(重新投影誤差), the camera matrix(相機的內參矩陣), distortion coefficients(相機畸變參數), rotation(旋轉) and translation(平移) vectors
retL, mtxL, distL, rvecsL, tvecsL = cv.calibrateCamera(objpoints, imgpointsL, grayL.shape[::-1], None, None)
heightL, widthL, channelsL = imgL.shape
# 回傳優化後的相機參數、圖像中不畸變的區域
newCameraMatrixL, roi_L = cv.getOptimalNewCameraMatrix(mtxL, distL, (widthL, heightL), 1, (widthL, heightL))

retR, mtxR, distR, rvecsR, tvecsR = cv.calibrateCamera(objpoints, imgpointsR, grayR.shape[::-1], None, None)
heightR, widthR, channelsR = imgR.shape
newCameraMatrixR, roi_R = cv.getOptimalNewCameraMatrix(mtxR, distR, (widthR, heightR), 1, (widthR, heightR))


In [8]:
header = ['Rotation','Translation', 'ProjectionLeft', 'ProjectionRight']

flags = 0
flags = cv.CALIB_FIX_INTRINSIC

criteria_stereo = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0001)

# 回傳相機的內參矩陣、畸變量、右相相對作相旋轉矩陣、右相相對作相旋轉矩陣、本徵矩陣、基本矩陣
retStereo, newCameraMatrixL, distL, newCameraMatrixR, distR, R, T, E, F = cv.stereoCalibrate(objpoints, imgpointsL, imgpointsR, mtxL, distL, mtxR, distR, grayL.shape[::-1], criteria_stereo, flags)

print(retStereo)
print("\nCamera matrix left: ")
print(newCameraMatrixL)
print("\ndistCoeffs left: ")
print(distL)
print("\nCamera matrix right: ")
print(newCameraMatrixR)
print("\ndistCoeffs right: ")
print(distR)
print("\nR: ")
print(R)
print("\nT: ")
print(T)
print("\nE: ")
print(E)
print("\nF: ")
print(F)

3.838934331389274

Camera matrix left: 
[[2.19798847e+03 0.00000000e+00 2.01642439e+03]
 [0.00000000e+00 2.18887589e+03 1.36446330e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

distCoeffs left: 
[[ 0.11634331 -0.12813081 -0.00768239  0.00134168 -0.12710841]]

Camera matrix right: 
[[2.74297455e+03 0.00000000e+00 2.03780197e+03]
 [0.00000000e+00 2.75152191e+03 1.34798421e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

distCoeffs right: 
[[ 0.0916629   0.486825   -0.01211016  0.00605343 -2.71630157]]

R: 
[[ 0.99390264 -0.01778127  0.10881805]
 [ 0.02133577  0.99927323 -0.03158785]
 [-0.10817729  0.03371697  0.99355968]]

T: 
[[-3.0935215 ]
 [ 0.50779967]
 [ 4.27185594]]

E: 
[[-0.14607571 -4.25162983  0.63946804]
 [ 3.91116013  0.02834514  3.53845325]
 [-0.57070609 -3.0822439   0.04245993]]

F: 
[[ 1.04256157e-06  3.04707687e-05 -5.37100398e-02]
 [-2.78277501e-05 -2.02514156e-07  1.05241753e-03]
 [ 4.65595296e-02 -1.22820334e-03  1.00000000e+00]]


In [11]:
import cv2
import numpy as np

def cat2images(limg, rimg):
    HEIGHT = limg.shape[0]
    WIDTH = limg.shape[1]
    imgcat = np.zeros((HEIGHT, WIDTH*2+20,3))
    imgcat[:,:WIDTH,:] = limg
    imgcat[:,-WIDTH:,:] = rimg
    for i in range(int(HEIGHT / 32)):
        imgcat[i*32,:,:] = 255 
    return imgcat


left_image = cv2.imread("left/IMG-0010.jpg")
right_image = cv2.imread("right/IMG-0015.jpg")

imgcat_source = cat2images(left_image,right_image)
HEIGHT = left_image.shape[0]
WIDTH = left_image.shape[1]
cv2.imwrite('imgcat_source.jpg', imgcat_source)

# 前面標定得到的cameraMatrix1
camera_matrix0 = np.array([[2.19798847e+03, 0.00000000e+00, 2.01642439e+03], 
                           [0.00000000e+00, 2.18887589e+03, 1.36446330e+03], 
                           [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]
                        ) .reshape((3,3)) 
# 前面標定得到的distCoeffs1
distortion0 = np.array([[0.11634331, -0.12813081, -0.00768239, 0.00134168, -0.12710841]])

# 前面標定得到的cameraMatrix2
camera_matrix1 = np.array([[2.74297455e+03, 0.00000000e+00, 2.03780197e+03], 
                           [0.00000000e+00, 2.75152191e+03, 1.34798421e+03], 
                           [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]
                        ) .reshape((3,3)) 
# 前面標定得到的distCoeffs2
distortion1 = np.array([[0.0916629, 0.486825, -0.01211016, 0.00605343, -2.71630157]])

# 前面標定得到的R
R = np.array([[ 0.99390264, -0.01778127,  0.10881805], 
              [ 0.02133577,  0.99927323, -0.03158785], 
              [-0.10817729,  0.03371697,  0.99355968]]
            ) 
# 前面標定得到的T
T = np.array([[-3.0935215 ], 
              [ 0.50779967], 
              [ 4.27185594]])


# 計算旋轉矩陣(R_l, R_r)和投影矩陣(P_l, P_r)
(R_l, R_r, P_l, P_r, Q, validPixROI1, validPixROI2) = cv2.stereoRectify(camera_matrix0, distortion0, camera_matrix1, distortion1, np.array([WIDTH,HEIGHT]), R, T)

# 左
# 產生映射表(計算無畸變、修正轉換關係)
(map1, map2) = cv2.initUndistortRectifyMap(camera_matrix0, distortion0, R_l, P_l, np.array([WIDTH,HEIGHT]), cv2.CV_32FC1) 
# 執行映射(將一張影像中某位置的像素放置到另一個圖片指定位置)
rect_left_image = cv2.remap(left_image, map1, map2, cv2.INTER_CUBIC) #重映射

# 右
(map1, map2) = cv2.initUndistortRectifyMap(camera_matrix1, distortion1, R_r, P_r, np.array([WIDTH,HEIGHT]), cv2.CV_32FC1)
rect_right_image = cv2.remap(right_image, map1, map2, cv2.INTER_CUBIC)

imgcat_out = cat2images(rect_left_image,rect_right_image)
cv2.imwrite('imgcat_out.jpg', imgcat_out)


True